In [142]:
import pandas as pd 
import numpy as np

In [143]:
df = pd.read_csv('Walmart.csv')
print(df.head())

   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010    1643690.90             0        42.31       2.572   
1      1  12-02-2010    1641957.44             1        38.51       2.548   
2      1  19-02-2010    1611968.17             0        39.93       2.514   
3      1  26-02-2010    1409727.59             0        46.63       2.561   
4      1  05-03-2010    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


In [144]:
target_col = 'Weekly_Sales'
num_cols = [ 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
cat_cols = ['Store', 'Holiday_Flag']
feature_cols = cat_cols + num_cols
# может перевести номер филиала в категориальный признак ?

# что делать с DATE numerical? categorical?

# Divide on train and test

In [145]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


In [146]:
one_hot_encoded = pd.get_dummies(df, columns=cat_cols)
df = one_hot_encoded
#df = pd.concat([df, one_hot_encoded], axis=1)
#df.drop(columns=cat_cols, axis=1, inplace= True)
#print(df)

In [147]:
#dataset = Dataset(X_tensor, y_tensor)

y = df[target_col]
X = df.drop(columns=target_col, axis=1)

In [148]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [149]:
mean = X_train[num_cols].mean()
std = X_train[num_cols].std()

# Standardize numerical features
X_train[num_cols] = (X_train[num_cols] - mean) / std
X_test[num_cols] = (X_test[num_cols] - mean) / std

In [10]:
#print(X_test)

In [150]:
X_train.drop(columns='Date', axis=1, inplace= True)
X_train = torch.tensor(X_train.values, dtype=torch.float32) 
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1) 

X_test.drop(columns='Date', axis=1, inplace= True)
X_test =  torch.tensor(X_test.values, dtype=torch.float32) 
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Sklearn LinearRegression 
проверка удалась, но теперь я не понимаю в чем проблема в нейронке

In [154]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

sklr = LinearRegression()
sklr.fit(X_train, y_train)

y_pred = sklr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Расчет R^2
r_squared = r2_score(y_test, y_pred)
print("R^2:", r_squared)

Mean Squared Error (MSE): 23934536000.0
R^2: 0.9247692991240415


# NN approach

# GPU

In [165]:
overallbest_hidden_seze = 5
overallbest_epo = 0
overallbest_lr = 0
overallbest_r2 = -9999999

for hidden_size in [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120]:
    model = nn.Sequential(
        nn.Linear(51, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, 1),
    )

    params = {'epo':[3, 10, 50, 100, 1000, 5000, 7000, 10_000],
            'lr':[10, 1, 0.1, 0.4, 0.01, 0.04, 0.001]}

    best_r2 = -99999
    best_epo = 0
    best_lr = 0
    #epochs = 10_000
    loss = nn.MSELoss()
    #optimiz = optim.Adam(model.parameters(), lr=0.1)


    model.to('cuda')
    X_test = X_test.to('cuda')
    X_train = X_train.to('cuda')
    y_test = y_test.to('cuda')
    y_train = y_train.to('cuda')

    num_samples = X_train.shape[0]
    train_loss = torch.zeros_like(y_train, device='cuda')
    train_pred = torch.zeros_like(y_train, device='cuda')

    for epochs in params['epo']:
        for lr in params['lr']:
            optimiz = optim.Adam(model.parameters(), lr=lr)
            for epoch in range(epochs):
                train_pred = model(X_train)
                train_loss = loss(train_pred, y_train)

                optimiz.zero_grad()
                train_loss.backward()
                optimiz.step()

            y_pred = model(X_test)

            r_squared = r2_score(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
            if(r_squared > best_r2):
                best_r2 = r_squared
                best_epo = epochs
                best_lr = lr

            #print('-'*40)
            #print("R^2:", r_squared)
            #print('epo:', epochs)
            #print('lr:', lr)

    print('#'*40)
    print('Best R2:', best_r2)
    print('Best params:')
    print('epo', best_epo)
    print('lr', best_lr)
    print('hiddel l size', hidden_size)
    if best_r2 > overallbest_r2:
        overallbest_r2 = best_r2
        overallbest_hidden_seze = hidden_size
        overallbest_epo = best_epo
        overallbest_lr = best_lr


print('#'*40)
print('#'*40)
print('#'*40)
print('Best R2:', overallbest_r2)
print('Best params:')
print('epo', overallbest_epo)
print('lr', overallbest_lr)
print('hiddel l size', overallbest_hidden_seze)     

########################################
Best R2: 0.9283009326389624
Best params:
epo 10000
lr 10
hiddel l size 5
########################################
Best R2: 0.9277044373273533
Best params:
epo 5000
lr 0.04
hiddel l size 10
########################################
Best R2: 0.9275681961495268
Best params:
epo 100
lr 0.001
hiddel l size 20
########################################
Best R2: 0.9308586227799842
Best params:
epo 5000
lr 1
hiddel l size 30
########################################
Best R2: 0.9277740845650989
Best params:
epo 1000
lr 10
hiddel l size 40
########################################
Best R2: 0.9276007102834174
Best params:
epo 100
lr 10
hiddel l size 50
########################################
Best R2: 0.9280220028941101
Best params:
epo 1000
lr 0.4
hiddel l size 60
########################################
Best R2: 0.9306941266103878
Best params:
epo 5000
lr 10
hiddel l size 70
########################################
Best R2: 0.9278286081672832
Best params:
epo

In [139]:
model = nn.Sequential(
    nn.Linear(51, 60),
    nn.ReLU(),
    nn.Linear(60, 1)

)

params = {'epo':[3, 10, 50, 100, 1000, 5000, 7000, 10_000],
          'lr':[10, 1, 0.1, 0.4, 0.01, 0.04, 0.001]}

best_r2 = -99999
best_epo = 0
best_lr = 0
#epochs = 10_000
loss = nn.MSELoss()
#optimiz = optim.Adam(model.parameters(), lr=0.1)


model.to('cuda')
X_test = X_test.to('cuda')
X_train = X_train.to('cuda')
y_test = y_test.to('cuda')
y_train = y_train.to('cuda')

num_samples = X_train.shape[0]
train_loss = torch.zeros_like(y_train, device='cuda')
train_pred = torch.zeros_like(y_train, device='cuda')

for epochs in params['epo']:
    for lr in params['lr']:
        optimiz = optim.Adam(model.parameters(), lr=lr)
        for epoch in range(epochs):
            train_pred = model(X_train)
            train_loss = loss(train_pred, y_train)

            optimiz.zero_grad()
            train_loss.backward()
            optimiz.step()

        y_pred = model(X_test)

        r_squared = r2_score(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
        if(r_squared > best_r2):
            best_r2 = r_squared
            best_epo = epochs
            best_lr = lr

        print('-'*40)
        print("R^2:", r_squared)
        print('epo:', epochs)
        print('lr:', lr)

print('#'*40)
print('Best R2:', best_r2)
print('Best params:')
print('epo', best_epo)
print('lr', best_lr)
        

----------------------------------------
R^2: -2.8920334371267775
epo: 3
lr: 10
----------------------------------------
R^2: -2.7571513291371352
epo: 3
lr: 1
----------------------------------------
R^2: -2.742787499536081
epo: 3
lr: 0.1
----------------------------------------
R^2: -2.6837605843576977
epo: 3
lr: 0.4
----------------------------------------
R^2: -2.682254396533711
epo: 3
lr: 0.01
----------------------------------------
R^2: -2.6762142165569633
epo: 3
lr: 0.04
----------------------------------------
R^2: -2.676062943461138
epo: 3
lr: 0.001
----------------------------------------
R^2: 0.13225289496424053
epo: 10
lr: 10
----------------------------------------
R^2: 0.5494369636387185
epo: 10
lr: 1
----------------------------------------
R^2: 0.5670074988928295
epo: 10
lr: 0.1
----------------------------------------
R^2: 0.6169229599135093
epo: 10
lr: 0.4
----------------------------------------
R^2: 0.6181442416260343
epo: 10
lr: 0.01
-------------------------------

# CPU

In [122]:
model = nn.Sequential(
    nn.Linear(51, 60),
    nn.ReLU(),
    nn.Linear(60, 1)
)

epochs = 10_000
batch_size = 256
loss = nn.MSELoss()
optimiz = optim.Adam(model.parameters(), lr=0.1)
mydevice = 'cpu'


model.to(mydevice)
X_test = X_test.to(mydevice)
X_train = X_train.to(mydevice)
y_test = y_test.to(mydevice)
y_train = y_train.to(mydevice)

num_samples = X_train.shape[0]
train_loss = torch.zeros_like(y_train, device=mydevice)
train_pred = torch.zeros_like(y_train, device=mydevice)
for epoch in range(epochs):
    train_pred = model(X_train)
    train_loss = loss(train_pred, y_train)

    optimiz.zero_grad()
    train_loss.backward()
    optimiz.step()

y_pred = model(X_test)

r_squared = r2_score(y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("R^2:", r_squared)

R^2: 0.9291178999855932
